In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
from src.preparation import fetch_dataset
from src.modules.visualizer import do_heatmap



In [3]:

import tensorflow as tf
from tensorflow.compat.v1.math import confusion_matrix
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.models import load_model
from src.processing import make_tf_dataset, split_train_test, dataset_to_array

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt



In [17]:
model_af = load_model('../notebook/model_a1_f1_100.h5')
model_ag = load_model('../notebook/model_a1_g1_100.h5')
model_gf = load_model('../notebook/model_f1_g1_100.h5')
model_all = load_model('../notebook/model_100.h5')

In [19]:
dataset_a = fetch_dataset('dataset_a1_slim.pkl')
dataset_f = fetch_dataset('dataset_f1_slim.pkl')
dataset_g = fetch_dataset('dataset_g1_slim.pkl')
dataset_all = fetch_dataset('dataset_all_slim.pkl')

In [6]:
# This is the size of the window that is fed into the DNN
window_size = 15
# The number of the features present in the dataset
num_of_features = 42
# Number of distinct labels in the output
label_length = 4
# Hyperparameter that defines the number of samples to work through
# before updating the internal model parameters.
batch_size = 25
# Epochs
ep = 100

In [15]:



def list_to_num(el):
    if el == [1,0,0,0]:
        return 0
    elif el == [0,1,0,0]:
        return 1
    elif el == [0,0,1,0]:
        return 2
    elif el == [0,0,0,1]:
        return 3
    else:
        print('Error',el)
        
def test_model(md, dt):
    print('\n')
    data_test = make_tf_dataset(dt, window_size, num_of_features, label_length ).batch(batch_size)
    results = md.evaluate(data_test)
    
    ph = []
    predictions = model_af.predict(data_test)
    for pred in predictions:
        i = list(pred).index(max(list(pred)))
        ph.append(i)
    # Ground truth
    fm , labels = dataset_to_array(dt)
    # Padding
    ph = ph + [3 for i in range (0, len(labels) - len(ph))]
    
    # -----------------
    labels = [list_to_num(list(x)) for x in labels]
    # CM
    cm = confusion_matrix(labels, ph)
    print('\n',cm)

In [16]:
test_model(model_af, dataset_g)
test_model(model_ag, dataset_f)
test_model(model_gf, dataset_a)



    725/Unknown - 6s 9ms/step - loss: 0.1132 - accuracy: 0.9773
 [[ 6666   256    24   122]
 [   40 10410     1   330]
 [   11     7   182    66]
 [    0     0     0     0]]


   2283/Unknown - 20s 9ms/step - loss: 0.0876 - accuracy: 0.9785
 [[12652   168   133   408]
 [   99 21849     9   526]
 [   99    32  3065   663]
 [  251   285   251 16589]]


    340/Unknown - 3s 10ms/step - loss: 0.0263 - accuracy: 0.9968
 [[3511  853   28  276]
 [  17 3287    0  527]
 [   0    0    0    0]
 [   0    0    0    0]]


In [20]:
test = dataset_all[-600:]

In [21]:
test_model(model_all, test)



   1703/Unknown - 15s 9ms/step - loss: 0.4251 - accuracy: 0.9368
 [[11390   609    80   428]
 [   59 17804     4   695]
 [   44    33  1715   296]
 [  132   135   162  8988]]
